# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

El caso de uso que se busca cubrir es el detectar si una jugada termina en home run o no cuando la bola ya fue bateada. Solo se va a poder usar dicho modelo para predecir el resultado de una jugada si se tienen los datos del lanzamiento y el bateo. Su principal uso está relacionado con el análisis de los factores que afectan a que una jugada sea home run.

# Configuración inicial

In [ ]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns
import warnings, time
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import QuantileTransformer
warnings.filterwarnings('ignore')
import graphviz
from sklearn.tree import export_graphviz
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [ ]:
# Arreglamos el dataset según lo establecido en el TP1

# Importamos train.csv y park_dimensions.csv, los unimos utilizando la variable "park"
entrenamiento = pd.read_csv('./train.csv')
estadio = pd.read_csv('./park_dimensions.csv')
completa=entrenamiento.merge(estadio, on="park", how="left")

# Desechamos las variables no utilizadas
completa = completa.drop(['park','bip_id','batter_id','pitcher_id'],axis=1)

# Asignamos nuevos nombres a las columnas
renamed_columns = {'NAME': 'name', 'Cover': 'cover', 'LF_Dim': 'lf_dim', 'CF_Dim':'cf_dim',
                   'RF_Dim': 'rf_dim', 'LF_W': 'lf_w', 'CF_W': 'cf_w', 'RF_W': 'rf_w'
                  }
completa.rename(columns=renamed_columns, inplace=True)

# Convertir columna "game_date" de tipo object/string, a datetime
completa['game_date'] = pd.to_datetime(completa['game_date'])

# Eliminar datos filas con datos nulos en bb_type
completa = completa[~completa.bb_type.isnull()]

# Delimitación de conjuntos
completa.isnull().sum()

# Crear df a la que se le va aplicar feature engineering
completa_fe = completa
columnas_string=['game_date', 'batter_team','batter_name','pitcher_name','name']
completa_fe = completa_fe.drop(columnas_string,axis=1)

# Eliminar columnas cuyos valores sean cadenas
columnas_string=['home_team','away_team', 'game_date', 'batter_team','batter_name','pitcher_name','name']
completa = completa.drop(columnas_string,axis=1)

## División del dataset

In [ ]:
# Dividimos el dataset en train (60%), test (20%) y validation (20%)
from sklearn.model_selection import train_test_split

train, not_train = train_test_split(completa, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

In [ ]:
# Definimos el mapper. Recibe una lista de (columna/s, transformers)

mapper = DataFrameMapper([ 
    (['is_batter_lefty'], None),
    (['is_pitcher_lefty'], None),
    (['bb_type'], [OneHotEncoder()]),
    (['bearing'], [OneHotEncoder()]),
    (['pitch_name'], [OneHotEncoder()]),
    (['inning'], [StandardScaler()]),
    (['outs_when_up'], None),
    (['balls'], [StandardScaler()]),
    (['strikes'], [StandardScaler()]),
    (['plate_x'], [StandardScaler()]),
    (['plate_z'], [StandardScaler()]),
    (['pitch_mph'], [StandardScaler()]),
    (['launch_speed'], [StandardScaler()]),
    (['launch_angle'], [StandardScaler()]),
    (['cover'], [OneHotEncoder()]),
    (['lf_dim'], [StandardScaler()]),
    (['cf_dim'], [StandardScaler()]),
    (['rf_dim'], [StandardScaler()]),
    (['lf_w'], [StandardScaler()]),
    (['cf_w'], [StandardScaler()]),
    (['rf_w'], [StandardScaler()]),
])

train_nor=train
mapper.fit(train_nor)

# Selección de métrica

Decidimos utilizar Recall y Precision, debido a que los valores de la variable de salida no se encuentran balanceados.
- Precision nos permite comprender el rendimiento de un clasificador con respecto a los falsos positivos (de los que clasificamos como Foo, qué porcentaje era realmente Foo).
- Recall nos permite comprender el rendimiento de un clasificador con respecto a falsos negativos (de todos los Foo que había, qué porcentaje era realmente Foo).

Al utilizar las dos, no solo ponderamos que el modelo tenga una precisión aceptable, sino que también consideramos cuántos casos está encontrando realmente (ya que, el modelo como tal puede no resultar tan útil si lo hace con muy pocos).

# Aplicaciones de featuring engineering

Nuestro dataset posee algunas variables a las que podemos aplicarles "feature engineering". Decidimos crear "division_home" y "division_away" a partir de los datos en las variables "home_team" y "away_team", de las cuales extraeremos la región a las que pertenecen los equipos en cuestión.

Por otra parte, aplicaremos la técnica de "Quantile Transformation" para ajustar a una distribución normal aquellas variables de entrada que presentan otro tipo de distribución, eliminando los valores atípicos.

Aplicamos técnicas de preprocesado para mejorar la representación de los datos como OneHotEncoder y StandardImputer, y eliminamos los valores nulos utilizando SimpleImputer.

## Extraer features a partir de otras

In [ ]:
# Regiones
east = ['TB','BAL','BOS','TOR','NYY','ATL','MIA','PHI','NYM','WSH']
central = ['MIN','CLE','DET','CWS','KC','PIT','MIL','CHC', 'CIN', 'STL']
west = ['TEX','LAA','HOU','SEA','OAK','LAD','SD','SF','COL','ARI']

def division_h(row):
    if east.count(row['home_team']) > 0:
        return 'east'
    else:
        if central.count(row['home_team']) > 0:
            return 'central'
        else:
            return 'west'
        
def division_a(row):
    if east.count(row['away_team']) > 0:
        return 'east'
    else:
        if central.count(row['away_team']) > 0:
            return 'central'
        else:
            return 'west'

# Nuevas features que almacenan la región del equipo local y visitante
division_home = completa_fe.apply(division_h, axis=1)
division_away = completa_fe.apply(division_a, axis=1)
completa_fe["division_home"] = division_home
completa_fe["division_away"] = division_away

# Eliminar features que ya no son necesarias
columnas_string=['home_team','away_team']
completa_fe = completa_fe.drop(columnas_string,axis=1)

## Aplicación de QuantileTransformer

Las features candidatas para aplicar "Quantile Transformation" son "plate_x", "plate_z", "pitch_mph", "launch_speed" y "launch_angle".

In [ ]:
# Función para graficar variables, antes y después de aplicar alguna transformación
def plots(df, col, t):
    plt.figure(figsize=(12,4))
    
    plt.subplot(121)
    sns.kdeplot(df[col])
    plt.title('Antes de aplicar ' + str(t).split('(')[0])

    plt.subplot(122)
    p1 = t.fit_transform(df[[col]]).flatten()
    sns.kdeplot(p1)
    plt.title('Después de aplicar ' + str(t).split('(')[0])

# Gráficas de variables, antes y después de aplicar Quantile Transformation
columns_qt = ['plate_x', 'plate_z', 'pitch_mph', 'launch_speed', 'launch_angle']
for col in columns_qt:
    plots(completa_fe, col, QuantileTransformer(output_distribution='normal'))

Como podemos visualizar en las gráficas, existen variables que ya presentan una distribución normal, mientras que otras presentan otro tipo de distribución. Es por ello, que decidimos aplicar la técnica en aquellas variables que lo requieren, como pitch_mph y launch_speed.

In [ ]:
qt = QuantileTransformer(output_distribution='normal')

# Creamos columnas con la técnica aplicada
completa_fe['pitch_mph_qt'] = qt.fit_transform(completa_fe.pitch_mph.to_frame())
completa_fe['launch_speed_qt'] = qt.fit_transform(completa_fe.launch_speed.to_frame())

# Eliminar features que ya no son necesarias
completa_fe = completa_fe.drop(['pitch_mph', 'launch_speed'],axis=1)

## División del dataset con feature engineering

In [ ]:
# Dividimos el dataset aplicando feature engine en train (60%), test (20%) y validation (20%)
train_fe, not_train_fe = train_test_split(completa_fe, test_size=0.4, random_state=42)
validation_fe, test_fe = train_test_split(not_train_fe, test_size=0.5, random_state=42)

train_fe.shape, validation_fe.shape, test_fe.shape

In [ ]:
# Definimos el mapper. Recibe una lista de (columna/s, transformers)
mapper_fe = DataFrameMapper([ 
    (['is_batter_lefty'], None), 
    (['is_pitcher_lefty'], None), 
    (['bb_type'], [OneHotEncoder()]), 
    (['bearing'], [OneHotEncoder()]), 
    (['pitch_name'], [OneHotEncoder()]), 
    (['inning'], [StandardScaler()]), 
    (['outs_when_up'], None), 
    (['balls'], [StandardScaler()]), 
    (['strikes'], [StandardScaler()]), 
    (['plate_x'], [StandardScaler()]), 
    (['plate_z'], [StandardScaler()]),    
    (['launch_angle'], [StandardScaler()]),
    (['cover'], [OneHotEncoder()]),
    (['lf_dim'], [StandardScaler()]),
    (['cf_dim'], [StandardScaler()]),
    (['rf_dim'], [StandardScaler()]),
    (['lf_w'], [StandardScaler()]),
    (['cf_w'], [StandardScaler()]),
    (['rf_w'], [StandardScaler()]),
    (["division_home"], [OneHotEncoder()]),
    (["division_away"], [OneHotEncoder()]),
    (['pitch_mph_qt'], [StandardScaler()]),
    (['launch_speed_qt'], [StandardScaler()]),
])

mapper_fe.fit(train_fe)

# Modelos a utilizar

Los modelos que vamos a utilizar van a ser:
 - K-Nearest Neighbors
 - Árboles de decisión
 - Random Forest
 - Gradient Boost
 
  Por cada modelo, se debe entrenarlo y realizar una exploración de hiper-parámetros mediante una búsqueda en grilla. Evaluar el comportamiento de cada modelo con los hiper-parámetros que mejores resultados ofrecen. En caso de ser posible, aporte conclusiones respecto a dicha comparación.

In [ ]:
# Función para evaluar modelos (sin aplicar feature engineering)
def evaluate_model(model, set_names=('train_nor', 'validation'), title='', show_cm=False):
    if title:
        display(title)
    
    final_metrics = {
        'Precision': [],
        'Recall': [],
        'F1': [],        
    }
        
    for i, set_name in enumerate(set_names):
        assert set_name in ['train_nor', 'validation', 'test']
        set_data = globals()[set_name]

        y = set_data.is_home_run
        y_pred = model.predict(set_data)
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

In [ ]:
# Función para evaluar modelos (aplicando feature engineering)    
def evaluate_model_fe(model, set_names=('train_fe', 'validation_fe'), title='', show_cm=False):
    if title:
        display(title)
        
    final_metrics = {
        'Precision': [],
        'Recall': [],
        'F1': [],        
    }
        
    for i, set_name in enumerate(set_names):
        assert set_name in ['train_fe', 'validation_fe', 'test_fe']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.is_home_run
        y_pred = model.predict(set_data)
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))

In [ ]:
# Gráfico del árbol
def graph_tree(tree, col_names):
    graph_data = export_graphviz(
        tree, 
        out_file=None, 
        feature_names=col_names,  
        class_names=['No es home run', 'Home run'],  
        filled=True, 
        rounded=True,  
        special_characters=True,
    )
    graph = graphviz.Source(graph_data)  
    return graph

## K-Nearest Neighbors

### Exploración de hiper-parámetros

In [ ]:
"""
k_range = list(range(2, 15))
knn = KNeighborsClassifier()
param_grid = dict(n_neighbors=k_range)
clf = GridSearchCV(knn, param_grid, scoring='f1', verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', clf),
])

gs_pipe.fit(train_nor, train_nor.is_home_run)
clf.best_score_, clf.best_params_
"""

# (0.12442273425645493, {'n_neighbors': 3})


### Sin feature engineering

In [ ]:
# Entrenamiento
k=3
knn_model = Pipeline([
    ('mapper', mapper),
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', KNeighborsClassifier(n_neighbors=k)),
])
inicio = time.time()
knn_model.fit(train_nor, train_nor.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model(knn_model, title='KNN')

### Con feature engineering

In [ ]:
# Entrenamiento
k=3
knn_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', KNeighborsClassifier(n_neighbors=k)),
])
inicio = time.time()
knn_model_fe.fit(train_fe, train_fe.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model_fe(knn_model_fe, title='KNN (aplicando feature engineering)')

### Conclusión

A partir de los resultados, podemos determinar que:

- El modelo (con la configuración planteada) es capaz de encontrar el 11% de las jugadas que terminaron en home run.
- Debido a la diferencia entre Precision y Recall, la configuración usada para este modelo nos devuelve un F1 de aproximadamente 28%.
- Con lo anterior, concluimos que el modelo no detecta muy bien las jugadas que son home run, por lo que puede ser necesario que se continúe trabajando con los datos de entrada.
- En lo que respecta a la aplicación de feature engineering, los valores de las métricas se ven afectados positivamente al train.

## Árboles de decisión

### Exploración de hiper-parámetros

In [ ]:
"""
profundidad = list(range(4, 12))
param_grid = dict(max_depth=profundidad)
clf = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, scoring='f1', verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', clf),
])

gs_pipe.fit(train_nor, train_nor.is_home_run)
clf.best_score_, clf.best_params_
"""
# Out: (0.5030525830032249, {'max_depth': 5})

tree_params = {'max_depth': 5}

### Sin feature engineering

In [ ]:
tree_model_limit = DecisionTreeClassifier(**tree_params, random_state=42)

# Entrenamiento
dt_model = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', tree_model_limit),
])
inicio = time.time()
dt_model.fit(train_nor, train_nor.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model(dt_model, title='Árbol de decisión')

### Con feature engineering

In [ ]:
tree_model_limit_fe = DecisionTreeClassifier(**tree_params, random_state=42)

# Entrenamiento
dt_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', tree_model_limit_fe),
])
inicio = time.time()
dt_model_fe.fit(train_fe, train_fe.is_home_run)
fin = time.time()
dt_time_train = fin - inicio
print(f"Demora entrenamiento: {dt_time_train}s")

# Evaluación
evaluate_model_fe(dt_model_fe, title='Árbol de decisión (aplicando feature engineering)')

### Conclusión

In [ ]:
# Graficar el árbol de decisión entrenado
graph_tree(tree_model_limit, mapper.transformed_names_)

A partir de los resultados, podemos determinar que:
- El 72% de las ocasiones en las que el modelo predijo un home run, acertó.
- El modelo (con la configuración planteada) es capaz de encontrar el 39% de las jugadas que terminaron en home run.
- Debido a la diferencia entre Precision y Recall, la configuración usada para este modelo nos devuelve un F1 de aproximadamente 50%.
- Con lo anterior, concluimos que el modelo no detecta muy bien las jugadas que son home run, pero cuando lo hace es confiable.
- En lo que respecta a la aplicación de feature engineering, los valores de las métricas se ven afectados negativamente.
- El árbol no es excesivamente grande para visualizarse, por lo que resulta más simple de entender e interpretar a diferencia de otros modelos. Podemos visualizar que las variables que tienen más impacto en las predicciones son  launch_angle y launch_speed.

## Random Forest

### Exploración de hiper-parámetros

In [ ]:
"""
parameters = {'n_estimators': [100, 200], 
              'max_depth':[3, 5, 8],
              'max_features': [2, 5]}
clf = GridSearchCV(RandomForestClassifier(random_state=42), parameters, scoring='f1', verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', clf),
])

gs_pipe.fit(train_nor, train_nor.is_home_run)
clf.best_score_, clf.best_params_
"""
# Out: (0.25394347435005404, {'max_depth': 8, 'max_features': 5, 'n_estimators': 200})

rf_params = {'max_depth': 8,
            'max_features': 5,
            'n_estimators': 200
            }

### Sin feature engineering

In [ ]:
forest_model = RandomForestClassifier(**rf_params, random_state=42)

# Entrenamiento
rf_model = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', forest_model),
])
inicio = time.time()
rf_model.fit(train_nor, train_nor.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model(rf_model, title='Random Forest')

### Con feature engineering

In [ ]:
forest_model_fe = RandomForestClassifier(**rf_params, random_state=42)

# Entrenamiento
rf_model_fe = Pipeline([
    ('mapper', mapper_fe),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', forest_model_fe),
])
inicio = time.time()
rf_model_fe.fit(train_fe, train_fe.is_home_run)
fin = time.time()
rf_time_train = fin - inicio
print(f"Demora entrenamiento: {rf_time_train}s")

# Evaluación
evaluate_model_fe(rf_model_fe, title='Random Forest (aplicando feature engineering)')

### Conclusión

In [ ]:
# Graficar el árbol de decisión entrenado
graph_tree(forest_model.estimators_[0], mapper.transformed_names_)

A partir de los resultados, podemos determinar que:

- El 80% de las ocasiones en las que el modelo predijo un home run, acertó.
- El modelo (con la configuración planteada) es capaz de encontrar el 20% de las jugadas que terminaron en home run.
- Debido a la diferencia entre Precision y Recall, la configuración usada para este modelo nos devuelve un F1 de aproximadamente 31%.
- Con lo anterior, concluimos que el modelo no detecta muy bien las jugadas que son home run, pero cuando lo hace es muy confiable.
- En lo que respecta a la aplicación de feature engineering, los valores de las métricas se ven afectados negativamente en F1 y positivamente en Precision.
- El árbol es excesivamente grande y complejo, por lo que entenderlo e interpretarlo se dificulta. A pesar de lo anterior, podemos visualizar que las variables que tienen más impacto en las predicciones son  launch_angle, launch_speed y plate_x.

## Gradient Boosting

### Exploración de hiper-parámetros

In [ ]:
"""
e_range = [10, 50, 100, 200]
param_grid = dict(n_estimators=e_range)
clf = GridSearchCV(GradientBoostingClassifier(), param_grid, scoring= 'f1',refit=True,verbose=1)

gs_pipe = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', clf),
])

gs_pipe.fit(train_nor, train_nor.is_home_run)
clf.best_score_, clf.best_params_
"""
# Out: (0.5265112649454026, {'n_estimators': 100})

gb_params = {'n_estimators':100}

### Sin feature engineering

In [ ]:
gb_model = GradientBoostingClassifier(**gb_params, random_state=42)

# Entrenamiento
model_gb100 = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', gb_model)
])
inicio = time.time()
model_gb100.fit(train_nor, train_nor.is_home_run)
fin = time.time()
gb_time_train = fin - inicio
print(f"Demora entrenamiento: {gb_time_train}s")

# Evaluación
evaluate_model(model_gb100, title='Gradient Boosting con n_trees=100')

### Con feature engineering

In [ ]:
gb_model_fe = GradientBoostingClassifier(**gb_params, random_state=42)

# Entrenamiento
model_gb100_fe = Pipeline([
    ('mapper', mapper_fe),
    ('imputer', IterativeImputer(random_state=42)),
    ('classifier', gb_model_fe)
])
inicio = time.time()
model_gb100_fe.fit(train_fe, train_fe.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model_fe(model_gb100_fe, title='Gradient Boosting con n_trees=100 (aplicando feature engineering)')

### Conclusión

A partir de los resultados, podemos determinar que:

- El 71% de las ocasiones en las que el modelo predijo un home run, acertó.
- El modelo (con la configuración planteada) es capaz de encontrar el 41% de las jugadas que terminaron en home run.
- Debido a la diferencia entre Precision y Recall, la configuración usada para este modelo nos devuelve un F1 de aproximadamente 52%.
- Con lo anterior, concluimos que el modelo no detecta muy bien las jugadas que son home run, pero cuando lo hace es confiable.
- En lo que respecta a la aplicación de feature engineering, los valores de las métricas se ven afectados negativamente, por lo que que se podría mejorar en dicho aspecto.

# Técnicas de reducción de la dimensionalidad 

El análisis de componentes principales (PCA) es una técnica utilizada para describir un conjunto de datos en términos de nuevas variables no correlacionadas, permite hacer dimensionality reduction. Al algoritmo se le puede indicar en "n_components" el número de componentes nuevas que se van a crear o la varianza que se quiere obtener en los datos. En el último caso, el algoritmo seleccionará el número de componentes para conservar el porcentaje de la varianza.

## Ejemplo demostrativo

In [ ]:
mapper_pca = DataFrameMapper([ 
    (['is_batter_lefty'], None), 
    (['bb_type'], [OneHotEncoder()]),
    (['is_pitcher_lefty'], None), 
    (['bearing'], [OneHotEncoder()]), 
    (['inning'], [StandardScaler()]), 
    (['outs_when_up'], None), 
    (['balls'], [StandardScaler()]), 
    (['strikes'], [StandardScaler()]), 
    (['plate_x'], [StandardScaler()]),
    (['plate_z'], [StandardScaler()]),   
    (['launch_angle'], [StandardScaler()]),
    (['launch_speed'], [StandardScaler()]),
    (['pitch_mph'], [StandardScaler()]),
    (['cover'], [OneHotEncoder()]),
    (['lf_dim'], [StandardScaler()]),
    (['cf_dim'], [StandardScaler()]),
    (['rf_dim'], [StandardScaler()]),
    (['lf_w'], [StandardScaler()]),
    (['cf_w'], [StandardScaler()]),
    (['rf_w'], None),
])

mapper_pca.fit(train)

pipe_pca1 = Pipeline([
    ('mapper', mapper_pca),
    ('imputer', SimpleImputer(strategy='mean')),
])

# preparar los datos
pipe_pca1.fit(train)
train_pca=pipe_pca1.transform(train)
train_pca= pd.DataFrame(train_pca, columns=mapper_pca.transformed_names_)

#aplicar la técnica de redución de dimensionalidades
pca_pipe = PCA(n_components=2, svd_solver="auto", random_state=42)
pca_pipe.fit(train_pca)
results = pca_pipe.fit_transform(train_pca)

#armar el dataframe con los datos obtenidos de la aplicación de PCA
results_red=pd.DataFrame({'PCA1':results[:,0], 'PCA2':results[:,1], 'clase':train.is_home_run})

sns.barplot(x=["PCA1","PCA2"], y=pca_pipe.explained_variance_ratio_)
print(pca_pipe.explained_variance_ratio_)
print(pca_pipe.explained_variance_ratio_.sum())

En lo anterior, podemos ver que aplicando dicha técnica y seleccionando la formación de dos componenetes, obtenemos que los mismos van a representar el 72% de la información total del dataset. Para perder menos infomración se podría aumentar el número de componentes. Como consecuencia, con las componentes calculadas en el ejemplo, se podrían utilizar modelos que no funcionan bien con muchas dimensiones, se ahorraría tiempo y recurso de hardware algoritmos de machine learning que usen esta data y se podría visualizar más facilmente la información.

## Árboles de decisión con PCA

In [ ]:
tree_model_limit_pca = DecisionTreeClassifier(**tree_params, random_state=42)

# Entrenamiento
dt_model_pca = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('pca', PCA(n_components=0.95)),
    ('classifier', tree_model_limit_pca),
])
start = time.time()
dt_model_pca.fit(train_nor, train_nor.is_home_run)
stop = time.time()
print(f"Demora entrenamiento: {stop - start}s")

# Evaluación
evaluate_model(dt_model_pca, title='Árbol de decisión (aplicando PCA)')


## Random Forest con PCA

In [ ]:
forest_model_pca = RandomForestClassifier(**rf_params, random_state=42)

# Entrenamiento
rf_model_pca = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('pca', PCA(n_components=0.95)),
    ('classifier', forest_model_pca),
])
inicio = time.time()
rf_model_pca.fit(train_nor, train_nor.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model(rf_model_pca, title='Random Forest (aplicando PCA)')

## Gradient Boost con PCA

In [ ]:
gb_model_pca = GradientBoostingClassifier(**gb_params, random_state=42)

# Entrenamiento
model_gb100_pca = Pipeline([
    ('mapper', mapper),
    ('imputer', IterativeImputer(random_state=42)),
    ('pca', PCA(n_components=0.95)),
    ('classifier', gb_model_pca),
])
inicio = time.time()
model_gb100_pca.fit(train_nor, train_nor.is_home_run)
fin = time.time()
print(f"Demora entrenamiento: {fin - inicio}s")

# Evaluación
evaluate_model(model_gb100_pca, title='Gradient Boosting con n_trees=100 (aplicando PCA)')

## Conclusión

PCA permite resumir la información de un gran número de features en un número limitado de componentes, pero dichos componentes no suelen ser intuitivos. Comparandolo con los modelos anteriores que no aplican PCA, concluimos que en nuestro caso no mejora el rendimiento de la predicción, la razón seguramente se deba a la pérdida de información que implica el reducir la dimensión del conjunto de datos.

# Técnicas para evitar overfitting

En relación a las técnicas para evitar overfitting que aplicamos a lo largo del problema podemos mencionar que llevamos a cabo las siguientes acciones:
- Dividimos los datasets (con y sin feature engineering) en train, test y validation. Esto nos permite obtener una valoración de aciertos/fallos del modelo, y compararlos entre subconjuntos para detectar fácilmente overfitting/underfitting.
- Modificamos los hiperparámetros en los modelos utilizados para que no sobreentrene. Por ejemplo, al establecer una profundidad de árbol y, de esa forma, evitar que el árbol de decisión se ajuste perfectamente al set de entrenamiento.
- Una mayor cantidad de datos permite que el modelo pueda generalizar mejor, teniendo en cuenta más tipos de datos, en nuestro caso podríamos utilizar datasets de otras temporadas.
- Eliminamos datos que podían llegar a provocar sobreentrenamiento tales como ids y nombres propios de personas, equipos, etc.

# Selección del modelo

In [ ]:
evaluate_model(knn_model, title='KNN', set_names=('train_nor', 'validation','test'))
print(f"Demora entrenamiento: {dt_time_train}s")

In [ ]:
evaluate_model(dt_model, title='Árboles de decisión', set_names=('train_nor', 'validation','test'))
print(f"Demora entrenamiento: {dt_time_train}s")

In [ ]:
evaluate_model(rf_model, title='Random Forest', set_names=('train_nor', 'validation','test'))
print(f"Demora entrenamiento: {rf_time_train}s")

In [ ]:
evaluate_model(model_gb100, title='Gradient Boosting con n_trees=100', set_names=('train_nor', 'validation','test'))
print(f"Demora entrenamiento: {gb_time_train}s")

Los modelos candidatos son árboles de decisión y Gradient Boosting, ya que presentan los valores más altos en F1 (que considera tanto la Precision como el Recall) utilizando el conjunto de datos "validation". Además, los tiempos de entrenamiento de ambos son similares y la diferencia entre el rendimiento del conjunto "train" y "validation" no es tan amplia en ninguno de los dos casos. Al final, optamos por el modelo de árboles de decisión debido a que los mismos pueden resultar más simples e intuitivos para que el cliente pueda interpretarlos.

La métrica final que será informada la conseguimos a partir del conjunto de datos "test", para obtener un valor lo más realista posible. De esta forma, al momento de informar al cliente acerca del rendimiento del modelo, se le comunica que posee una precisión del 68% y que es capaz de identificar el 48% de los casos en que una jugada termina en home run.

# Diagramas de dispersión con aciertos y errores

In [ ]:
# Función para evaluar modelos (sin aplicar feature engineering)
def evaluate_model(model, set_names=('train_nor', 'validation'), title='', show_cm=False):
    if title:
        display(title)
    
    final_metrics = {
        'Precision': [],
        'Recall': [],
        'F1': [],        
    }
        
    for i, set_name in enumerate(set_names):
        assert set_name in ['train_nor', 'validation', 'test']
        set_data = globals()[set_name]

        y = set_data.is_home_run
        y_pred = model.predict(set_data)
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
    display(pd.DataFrame(final_metrics, index=set_names))
    return y_pred

y_pred=evaluate_model(dt_model, title='Árboles de decisión', set_names=('train_nor', 'validation','test'))

#Función para obtener valores correctos y errores
def comparar_resultados(row):
    if (row['is_home_run']) == 0 and (row['is_home_run'])!=(row['resultados_reales']):
        return 2
    else:
        if (row['is_home_run']) == 0:
            return 0
        else:
            if (row['is_home_run']) == 1 and (row['is_home_run'])!=(row['resultados_reales']):
                return 3
            else:
                return 1
            
validation["resultados_reales"]=y_pred
resultados_finales = validation.apply(comparar_resultados, axis=1)
validation["resultados_finales"]=resultados_finales

### Referencias

 - 0: Era False y dijo False
 - 1: Era True y dijo True
 - 2: Era False y dijo True
 - 3: Era True y dijo False

In [ ]:
sns.scatterplot(x=validation.launch_angle, y=validation.launch_speed, hue=validation.resultados_finales, data=validation)

En el gráfico podemos observar la relación entre los aciertos y los errores en la predicción de la variable de salida con relación a las variables launch_speed y launch_angle. Si bien no se ve tan claramente porque hay una gran cantidad de datos superpuestos, podemos decir que el algoritmo no pudo encontrar la relación entre los rangos de ambas variables, debido a que en la parte donde más positivos había no obtuvo buenas predicciones. Esto podría deberse a que priorizaba decir "False" debido al desbalance de los datos. Esto se podría solucionar asignando pesos a las clases.

In [ ]:
sns.scatterplot(x=validation.bb_type, y=validation.pitch_mph, hue=validation.resultados_finales, data=validation)

En el gráfico podemos observar la relación entre los aciertos y los errores en la predicción de la variable de salida con relación a las variables pitch_mph y bb_type. Como en el caso anterior, no se ven tan claramente todos los puntos porque hay una gran cantidad de datos superpuestos, pero podemos decir que el algoritmo no pudo encontrar tanto la relación entre las categorías de bb_type (tipo de bola bateada) que a una determinada pitch_mph (velocidad de bola lanzada) puede terminar en home run o no. Esto se hace visible por la aparición de un importante números de puntos "Era True y dijo False"